Stand: 09.08.2025

In [41]:
import pandas as pd


clean_sensors = pd.read_csv("../ws_compl_summer.csv")
clean_sensors.date = pd.to_datetime(clean_sensors.date)
clean_sensors = clean_sensors.loc[:, ["name", "date", "hour", "temperature"]]

# Start- und Enddatum festlegen
start = "2024-06-01"
end = "2024-08-31"

# Filtern
clean_sensors_summer = clean_sensors[
    (clean_sensors["date"] >= start) & (clean_sensors["date"] <= end)
]

clean_sensors_summer.info()
clean_sensors_summer

<class 'pandas.core.frame.DataFrame'>
Index: 24288 entries, 1224 to 40871
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         24288 non-null  object        
 1   date         24288 non-null  datetime64[ns]
 2   hour         24288 non-null  int64         
 3   temperature  24242 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 948.8+ KB


,name,date,hour,temperature
1224,Bodanplatz,2024-06-01,0,10.95
1225,Bodanplatz,2024-06-01,1,11.00
1226,Bodanplatz,2024-06-01,2,10.90
1227,Bodanplatz,2024-06-01,3,10.90
1228,Bodanplatz,2024-06-01,4,10.85
...,...,...,...,...
40867,Stephansplatz,2024-08-31,19,29.05
40868,Stephansplatz,2024-08-31,20,27.45
40869,Stephansplatz,2024-08-31,21,26.60
40870,Stephansplatz,2024-08-31,22,25.10


Heiße Tage auswerten - Tage im Sommer mit max. Wert > 30 °C

In [43]:
# 1) Tagesmaximum je Station/Tag
daily_max = clean_sensors_summer.groupby(["name", "date"], as_index=False)[
    "temperature"
].max()

# 2) Tage markieren, an denen > 30 °C erreicht wurden
daily_max["hot"] = daily_max["temperature"] > 30

# 3) Pro Station: heiße Tage zählen
hot_days_per_station = (
    daily_max.groupby("name")
    .agg(
        Hot_Days_Count=("hot", "sum"),
        Hottest_Day=("temperature", lambda x: round(x.max(), 1)),
    )
    .reset_index()
)

# 4) Datum des heißesten Tages pro Station ermitteln
hottest_dates = daily_max.loc[
    daily_max.groupby("name")["temperature"].idxmax(), ["name", "date"]
].rename(columns={"date": "Hottest_Day_Date"})

# 5) Zusammenführen
hot_days_per_station = hot_days_per_station.merge(hottest_dates, on="name")

# 6) Sortieren
hot_days_per_station.sort_values("Hot_Days_Count")

hot_days_per_station.to_csv("../hot_data.csv")